Assignment 2: Time Series Forcasting 

Dataset URL: https://www.kaggle.com/datasets/bobnau/daily-website-visitors

In [26]:
# Import necessary libraries for time series analysis
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit


In [57]:
# Add File Path of the CSV file here
# file_path = ""
file_path = "C:/Users/matt/Downloads/archive (3)/daily-website-visitors.csv"

# Reading the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

In [46]:
# Print column names to identify the correct date column
print(df.columns)


Index(['Row', 'Day', 'Day.Of.Week', 'Date', 'Page.Loads', 'Unique.Visits',
       'First.Time.Visits', 'Returning.Visits'],
      dtype='object')


# Pre Processing for Time Series Analysis:

In [ ]:
# Step 1: Import necessary libraries
from tbats import BATS
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# Assuming df['Unique.Visits'] is already cleaned and processed

# Step 2: Split the data into train and test sets (80% train, 20% test)
train_size = int(len(df) * 0.8)
train, test = df['Unique.Visits'][:train_size], df['Unique.Visits'][train_size:]

# Step 3: Initialize and fit the BATS model with weekly seasonality (7 days)
estimator = BATS(seasonal_periods=[7])  # Assuming weekly seasonality
bats_model = estimator.fit(train)

# Step 4: Forecast on the test set
forecast = bats_model.forecast(steps=len(test))

# Step 5: Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(test, forecast)
print(f"Mean Squared Error (MSE) on Test Set: {mse}")

# Step 6: Plot actual vs predicted values
plt.figure(figsize=(10,6))
plt.plot(test.index, test, label='Actual Unique Visits', color='blue')
plt.plot(test.index, forecast, label='Forecasted Unique Visits (BATS)', color='red', linestyle='--')
plt.title('BATS Model: Actual vs Forecasted Unique Visits')
plt.xlabel('Date')
plt.ylabel('Unique Visits')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Import required packages
from tbats import TBATS
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Preprocess the 'Unique.Visits' column by removing commas and converting to numeric
df['Unique.Visits'] = df['Unique.Visits'].str.replace(',', '').astype(float)

# Step 2: Split the data into train and test sets (80% train, 20% test)
train_size = int(len(df) * 0.8)
train, test = df['Unique.Visits'][:train_size], df['Unique.Visits'][train_size:]

# Step 3: Initialize and fit the TBATS model
estimator = TBATS(seasonal_periods=[7])  # Assuming weekly seasonality
tbats_model = estimator.fit(train)

# Step 4: Forecast on the test set
forecast = tbats_model.forecast(steps=len(test))

# Step 5: Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(test, forecast)
print(f"Mean Squared Error (MSE) on Test Set: {mse}")

# Step 6: Plot actual vs predicted values
plt.figure(figsize=(10,6))
plt.plot(test.index, test, label='Actual Unique Visits', color='blue')
plt.plot(test.index, forecast, label='Forecasted Unique Visits (TBATS)', color='red', linestyle='--')
plt.title('TBATS Model: Actual vs Forecasted Unique Visits')
plt.xlabel('Date')
plt.ylabel('Unique Visits')
plt.legend()
plt.grid(True)
plt.show()

